### summarize lasso results on Independent Simulation Scenarios for binary outcome

In [1]:
dir = '/panfs/panfs1.ucsd.edu/panscratch/lij014/Stability_2020/sim_data'

In [2]:
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

files = NULL
for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_independent_', paste('P', p, 'N', n, sep='_'), '.RData'))
}

In [3]:
length(files)

[1] 16

In [4]:
avg_FDR = NULL
table_toe = NULL
tmp_num_select = rep(0, length(files))
for (i in 1:length(files)){
    print(paste0('indx: ', i))
    load(paste0(dir, '/binary_update/ind_Lasso_binary_', i, '.RData')) 
    
    table_toe = rbind(table_toe, results_ind_lasso[c('n', 'p', 'rou', 'FP', 'FN', 'ROC', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_ind_lasso$Stab.table))
    
    # calculate FDR
    load(file_name, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_ind_lasso$Stab.table
    FDR = NULL # false positive rate
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}
table_toe = as.data.frame(table_toe)
table_toe$num_select = tmp_num_select
table_toe$FDR = round(avg_FDR,2)

[1] "indx: 1"
[1] "indx: 2"
[1] "indx: 3"
[1] "indx: 4"
[1] "indx: 5"
[1] "indx: 6"
[1] "indx: 7"
[1] "indx: 8"
[1] "indx: 9"
[1] "indx: 10"
[1] "indx: 11"
[1] "indx: 12"
[1] "indx: 13"
[1] "indx: 14"
[1] "indx: 15"
[1] "indx: 16"


In [5]:
head(table_toe)

,n,p,rou,FP,FN,ROC,Stab,num_select,FDR
,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<dbl>,<dbl>
1,50,50,0,3.69 ( 0.25 ),2.5 ( 0.12 ),0.74 ( 0.01 ),0.33,6.19,0.37
2,100,50,0,7.16 ( 0.37 ),0.64 ( 0.08 ),0.85 ( 0.01 ),0.35,11.52,0.48
3,500,50,0,5.21 ( 0.21 ),0 ( 0 ),0.93 ( 0 ),0.53,10.21,0.39
4,1000,50,0,2.25 ( 0.11 ),0 ( 0 ),0.93 ( 0 ),0.8,7.25,0.15
5,50,100,0,5.13 ( 0.34 ),2.93 ( 0.1 ),0.73 ( 0.01 ),0.25,7.20,0.50
6,100,100,0,8.22 ( 0.44 ),1.03 ( 0.09 ),0.83 ( 0.01 ),0.32,12.19,0.54


In [6]:
tail(table_toe)

,n,p,rou,FP,FN,ROC,Stab,num_select,FDR
,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<dbl>,<dbl>
11,500,500,0,23.83 ( 0.69 ),0 ( 0 ),0.92 ( 0 ),0.2,28.83,0.78
12,1000,500,0,11.38 ( 0.41 ),0 ( 0 ),0.93 ( 0 ),0.36,16.38,0.60
13,50,1000,0,9.77 ( 0.24 ),4.03 ( 0.08 ),0.67 ( 0.01 ),0.09,10.74,0.81
14,100,1000,0,11.91 ( 0.8 ),2.32 ( 0.1 ),0.78 ( 0.01 ),0.2,14.59,0.66
15,500,1000,0,30.84 ( 0.92 ),0 ( 0 ),0.91 ( 0 ),0.16,35.84,0.82
16,1000,1000,0,19.96 ( 0.61 ),0 ( 0 ),0.93 ( 0 ),0.24,24.96,0.74


In [7]:
# export result
result.table_toe <- apply(table_toe,2,as.character)
rownames(result.table_toe) = rownames(table_toe)
result.table_toe = as.data.frame(result.table_toe)

# extract numbers only for 'n' & 'p'
result.table_toe$n = tidyr::extract_numeric(result.table_toe$n)
result.table_toe$p = tidyr::extract_numeric(result.table_toe$p)
result.table_toe$ratio = result.table_toe$p / result.table_toe$n

result.table_toe = result.table_toe[c('n', 'p', 'rou', 'ratio', 'Stab', 'ROC', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_toe)[1:4] = c('N', 'P', 'Corr', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead

extract_numeric() is deprecated: please use readr::parse_number() instead



In [8]:
# convert interested measurements to be numeric
result.table_toe$Stab = as.numeric(as.character(result.table_toe$Stab))
result.table_toe$num_select = as.numeric(as.character(result.table_toe$num_select))

result.table_toe$ROC_mean = as.numeric(sub("\\(.*", "", result.table_toe$ROC))
result.table_toe$FP_mean = as.numeric(sub("\\(.*", "", result.table_toe$FP))
result.table_toe$FN_mean = as.numeric(sub("\\(.*", "", result.table_toe$FN))

In [9]:
# check whether missing values exists
result.table_toe[rowSums(is.na(result.table_toe)) > 0,]

N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>


In [10]:
head(result.table_toe)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
1,50,50,0,1.00,0.33,0.74 ( 0.01 ),3.69 ( 0.25 ),2.5 ( 0.12 ),6.19,0.37,0.74,3.69,2.50
2,100,50,0,0.50,0.35,0.85 ( 0.01 ),7.16 ( 0.37 ),0.64 ( 0.08 ),11.52,0.48,0.85,7.16,0.64
3,500,50,0,0.10,0.53,0.93 ( 0 ),5.21 ( 0.21 ),0 ( 0 ),10.21,0.39,0.93,5.21,0.00
4,1000,50,0,0.05,0.80,0.93 ( 0 ),2.25 ( 0.11 ),0 ( 0 ),7.25,0.15,0.93,2.25,0.00
5,50,100,0,2.00,0.25,0.73 ( 0.01 ),5.13 ( 0.34 ),2.93 ( 0.1 ),7.20,0.5,0.73,5.13,2.93
6,100,100,0,1.00,0.32,0.83 ( 0.01 ),8.22 ( 0.44 ),1.03 ( 0.09 ),12.19,0.54,0.83,8.22,1.03


In [11]:
tail(result.table_toe)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
11,500,500,0,1.0,0.20,0.92 ( 0 ),23.83 ( 0.69 ),0 ( 0 ),28.83,0.78,0.92,23.83,0.00
12,1000,500,0,0.5,0.36,0.93 ( 0 ),11.38 ( 0.41 ),0 ( 0 ),16.38,0.6,0.93,11.38,0.00
13,50,1000,0,20.0,0.09,0.67 ( 0.01 ),9.77 ( 0.24 ),4.03 ( 0.08 ),10.74,0.81,0.67,9.77,4.03
14,100,1000,0,10.0,0.20,0.78 ( 0.01 ),11.91 ( 0.8 ),2.32 ( 0.1 ),14.59,0.66,0.78,11.91,2.32
15,500,1000,0,2.0,0.16,0.91 ( 0 ),30.84 ( 0.92 ),0 ( 0 ),35.84,0.82,0.91,30.84,0.00
16,1000,1000,0,1.0,0.24,0.93 ( 0 ),19.96 ( 0.61 ),0 ( 0 ),24.96,0.74,0.93,19.96,0.00


In [12]:
result.table_toe

## export
write.table(result.table_toe, '../results_summary_bin/sim_ind_lasso_binary.txt', 
            sep='\t', row.names=F)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
1,50,50,0,1.00,0.33,0.74 ( 0.01 ),3.69 ( 0.25 ),2.5 ( 0.12 ),6.19,0.37,0.74,3.69,2.50
2,100,50,0,0.50,0.35,0.85 ( 0.01 ),7.16 ( 0.37 ),0.64 ( 0.08 ),11.52,0.48,0.85,7.16,0.64
3,500,50,0,0.10,0.53,0.93 ( 0 ),5.21 ( 0.21 ),0 ( 0 ),10.21,0.39,0.93,5.21,0.00
4,1000,50,0,0.05,0.80,0.93 ( 0 ),2.25 ( 0.11 ),0 ( 0 ),7.25,0.15,0.93,2.25,0.00
5,50,100,0,2.00,0.25,0.73 ( 0.01 ),5.13 ( 0.34 ),2.93 ( 0.1 ),7.20,0.5,0.73,5.13,2.93
6,100,100,0,1.00,0.32,0.83 ( 0.01 ),8.22 ( 0.44 ),1.03 ( 0.09 ),12.19,0.54,0.83,8.22,1.03
7,500,100,0,0.20,0.40,0.92 ( 0 ),8.89 ( 0.29 ),0 ( 0 ),13.89,0.55,0.92,8.89,0.00
8,1000,100,0,0.10,0.69,0.93 ( 0 ),3.53 ( 0.16 ),0 ( 0 ),8.53,0.27,0.93,3.53,0.00
9,50,500,0,10.00,0.13,0.7 ( 0.01 ),8.05 ( 0.28 ),3.74 ( 0.09 ),9.31,0.74,0.70,8.05,3.74
